In [1]:
import numpy as np
import pandas as pd
import random

In [5]:
def adaptive_asset_allocation(df, nlargest, volatility_window, return_window, portfolio_value): 
    window_returns = np.log(df.iloc[-1]) - np.log(df.iloc[0])
    nlargest = list(window_returns.nlargest(4).index)
    
    returns = df[nlargest].pct_change()
    returns_cov_normalized = returns[-volatility_window:].apply(lambda x: np.log(1+x)).cov()
    returns_corr_normalized = returns[-volatility_window:].apply(lambda x: np.log(1+x)).corr()
    returns_std = returns.apply(lambda x: np.log(1+x)).std()
    
    port_returns = []
    port_volatility = []
    port_weights = []

    num_assets = len(returns.columns)
    num_portfolios = 1000
    individual_rets = window_returns[nlargest]
    
    for port in range(num_portfolios): 
        weights = np.random.random(num_assets)
        weights = weights/np.sum(weights)
        port_weights.append(weights)

        rets = np.dot(weights, individual_rets)
        port_returns.append(rets)

        var = returns_cov_normalized.mul(weights, axis=0).mul(weights, axis=1).sum().sum()
        sd = np.sqrt(var)
        ann_sd = sd * np.sqrt(256)
        port_volatility.append(ann_sd)
        
    data = {'Returns': port_returns, 'Volatility': port_volatility}
    hover_data = []
    for counter, symbol in enumerate(nlargest): 
        data[symbol] = [w[counter] for w in port_weights]
        hover_data.append(symbol)

    portfolios_V1 = pd.DataFrame(data)
    
    min_var_portfolio = portfolios_V1.iloc[portfolios_V1['Volatility'].idxmin()]
    return min_var_portfolio[nlargest] * portfolio_value

In [6]:
from yahoo_fin.stock_info import *

basket = ['VTI', 'VXUS', 'VNQ', 'BND', 'GLD', 'EFA', 'TLT', 'IEF', 'QQQ']
df_data = {}
for stock_ticker in basket: 
    data = get_data(stock_ticker)
    adj_closed = data['adjclose'][-180:]
    df_data[stock_ticker] = adj_closed
    
df = pd.DataFrame(df_data)

adaptive_asset_allocation(df, 4, 20, 180, 35000)

QQQ      364.790149
VNQ    10220.549269
TLT    12396.880021
VTI    12017.780562
Name: 628, dtype: float64

In [4]:
df

,VTI,VXUS,VNQ,BND,GLD,EFA,TLT,IEF,QQQ
2021-03-17,206.817780,62.847801,90.926430,83.604019,163.509995,75.882347,133.862823,113.201889,320.765808
2021-03-18,203.053848,62.109573,90.004211,83.258598,162.559998,75.191795,132.507278,112.495689,310.940582
2021-03-19,203.311386,62.424549,88.777870,83.327682,163.240005,75.398956,133.328522,112.426064,312.036713
2021-03-22,204.410843,62.419613,89.405754,83.564537,163.000000,75.468010,134.822586,112.794083,317.888367
2021-03-23,202.083160,61.472370,89.278214,83.752068,161.809998,74.422318,136.039642,113.351097,316.501556
...,...,...,...,...,...,...,...,...,...
2021-11-23,240.399994,64.559998,109.760002,84.470001,167.279999,79.790001,144.500000,113.550003,397.480011
2021-11-24,241.210007,64.300003,111.089996,84.669998,167.119995,79.260002,146.820007,113.900002,398.760010
2021-11-26,235.990005,62.459999,108.000000,85.269997,166.850006,77.129997,150.529999,115.370003,391.200012
2021-11-29,238.210007,62.730000,108.800003,85.260002,166.619995,77.519997,149.320007,115.110001,399.690002
